# 2-D Diffusion Equation (Stationary)

## Introduction

Assuming that the temperature no longer changes with time, i.e., the temperature field is stationary, the heat conduction equation simplifies to a pure diffusion equation in the form of a so-called Poisson equation (radiogenic sources only):

$\begin{equation}
0 = \frac{\partial}{\partial{x}} \left(k_{x} \frac{\partial{T}}{\partial{x}}\right) + \frac{\partial}{\partial{y}} \left(k_{y} \frac{\partial{T}}{\partial{y}}\right) + \rho H_r,
\end{equation}$

where $\rho$ is the density, $k_{x,y}$ are the thermal conductivities in the $x$- and $y$-directions, and $H_r$ is the radiogenic heat production per unit mass [W/kg]. Let us simplify the equation further by assuming that the thermal parameters (here, in particular the thermal conductivity $k$) are **isotropic** and **constant**:

$\begin{equation}
0 = \left( \frac{\partial^2{T}}{\partial{x}^2} + \frac{\partial^2{T}}{\partial{y}^2} \right) + \frac{Q}{k},
\end{equation}$

where $Q = \rho H_r$ is the volumetric heat production rate [W/m³].


## The Problem

Assume that Equation (2) is defined in a 2D rectangular domain with a side length ($L$) of 4000 m and a depth ($H$) of 2000 m. Within the domain, a body with a high heat production rate is embedded, with corner coordinates (x, z) in km: (1900, 900), (2100, 900), (2100, 1100), and (1900, 1100). The heat production rate $Q$ is 0.3 W/m<sup>3</sup> inside the body and 0 outside. The thermal conductivity ($k$) is 6.5 W/(K m). As a boundary condition (Dirichlet), set T = 0 °C along the entire boundary.

<img src="./Figures/Exercise04_1.png" alt="drawing" width="450"/> <br>
**Fig. 1.** Model setup

The configuration described roughly represents the situation of high-level radioactive waste disposal in a salt dome. There, radioactive canisters with a diameter of 1 m are to be stored in 250 m deep boreholes at about 1 km depth. The average spacing of such boreholes is about 50 m across a width of 250 m and a length of 1–2 km. With a heat production of slightly less than 1 kW for a 1 m high canister, one arrives at approximately the above average heat production rate over the entire disposal area.

To solve the problem using Julia, we first need to define the required modules (`ExtendableSparse, Plots`) and submodules (`GeoModBox.HeatEquation.TwoD`):


In [ ]:
using ?

We now define the model dimensions ($L$, $H$) and physical parameters ($k$, $Q$):

In [ ]:
# Physical parameters --------------------------------------------------- #
P       = ( 
    L       =   ?,      # Length   [m]
    H       =   ?,      # Height   [m]
    k       =   ?,      # Thermal conductivity [W/m/K]
    # Define the region of the heat source
    Wcave   =   ?,      # Width of the region [m]
    Hcave   =   ?,      # Thickness [m]
    Dcave   =   ?,      # Depth of the center [m]
    Xcave   =   ?,      # x-position of the center [m]
    Q       =   ?       # Volumetric heat production rate [W/m³]
)
# ----------------------------------------------------------------------- #

Next, we define the number of grid points and the grid spacing:

In [ ]:
# Numerical parameters -------------------------------------------------- #
NC      = (
    x       =   ?,      # Grid points in x-direction
    y       =   ?       # Grid points in y-direction    
)
Δ       = (
    x       =   ?,      # Grid spacing in x-direction
    y       =   ?       # Grid spacing in y-direction
)
# ----------------------------------------------------------------------- #

With these, the numerical grid can be defined, along with the initial conditions of our problem:

In [ ]:
# Grid creation ---------------------------------------------------------- #
x       = (
    c       =   LinRange(?),
)
y       = (
    c       =   LinRange(?),
)
# ----------------------------------------------------------------------- #
# Initialization --------------------------------------------------------- #
D       = ( 
    Q       =   zeros( ? ),
    T       =   zeros( ? ),
)
# Define the anomaly region ---
for i = 1:NC.x, j = 1:NC.y
    ?
end
# ----------------------------------------------------------------------- #

## The Solution

### Discretization

To solve the problem numerically, we need to subdivide our model domain into a numerical grid. We assume that the temperature is defined at so-called *cell-centered* grid points (centroids) (see Fig. 2). To solve our problem, we also use so-called *ghost nodes* in our grid, which allow a correct incorporation of the boundary conditions.

#### Grid and Indexing

<img src="./Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>
**Fig. 2.** Staggered grid. The temperature is defined at the cell-centered grid points, the centroids (red circles). The gray circles outside the model domain are the ghost nodes.

The given staggered grid enables a so-called *conservative* finite difference approximation, where it is assumed that the heat flux $q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ is defined at the midpoint of the grid lines and the temperature is defined at the center of a grid cell (strictly speaking, the thermal conductivity is then also defined on the grid lines; however, since it is constant, we do not need to consider this here; for variable thermal parameters, the equation must be discretized somewhat differently). Moreover, using cell-centered temperature points in combination with *ghost nodes* allows us to incorporate boundary conditions relatively easily, while retaining the same order of accuracy as the central difference quotients in the interior of our model.

When indexing our grid points, we distinguish between *local* and *global* indices. The local index describes the position on the $i$,$j$ grid. The global index is a running index from 1 to $nc_x \cdot nc_y$ and corresponds to the number of equations, i.e., the total number of interior grid points. The global index is also used when assembling the coefficient matrix for our linear system of equations.

For each grid point, i.e., for each equation, a so-called numerical stencil specifies the positions of the grid points that are relevant for the respective equation. The coefficients for these grid points are then non-zero, while all others are zero. The nomenclature for the stencil often follows that of a compass, i.e., we have points to the south, west, center, east, and north. For each equation, the global index ($I$) of each stencil point is given by its position relative to the central point ($I^\textrm{C}$) of the stencil, i.e.:

$\begin{equation}\begin{split}
I^\textrm{S} & = I^\textrm{C} - nc_x,\\
I^\textrm{W} & = I^\textrm{C} - 1,\\
I^\textrm{C} & = I, \\
I^\textrm{E} & = I^\textrm{C} + 1,\\
I^\textrm{N} & = I^\textrm{C} + nc_x,
\end{split}\end{equation}$

where $nc_x$ is the number of horizontal centroids, $I^\textrm{C}$ is the central reference point, and $I^\textrm{S}, I^\textrm{W}, I^\textrm{E}, I^\textrm{N}$ are the points to the south, west, east, and north of it.

#### Finite Difference Approximation

We can now approximate the partial differential equation using our finite differences as:

$\begin{equation}
0 = \left( \frac{T_{I^\textrm{W}} - 2T_{I^\textrm{C}} + T_{I^\textrm{E}}}{\Delta{x}^2} + \frac{T_{I^\textrm{S}} - 2T_{I^\textrm{C}} + T_{I^\textrm{N}}}{\Delta{y}^2} \right) + \frac{Q}{k},
\end{equation}$

where $\Delta{x}$ and $\Delta{y}$ are the grid spacings in the $x$ and $y$ directions. Rearranging yields a linear system with five coefficients of the form:

$\begin{equation}
b T_{I^\textrm{S}} + aT_{I^\textrm{W}} - (2a + 2b) T_{I^\textrm{C}} + a T_{I^\textrm{E}} + b T_{I^\textrm{N}} = - \frac{Q_{I^\textrm{C}}}{k},
\end{equation}$

where $a = 1 / \Delta{x}^2$ and $b = 1 / \Delta{y}^2$.

### Boundary Conditions

The temperature at the *ghost nodes* is defined for *Dirichlet* and *Neumann* boundary conditions in the same way as in the 1-D [explicit](./02_1D_Heat_explicit.ipynb) or [implicit](./03_1D_Heat_implicit.ipynb) solution of the heat diffusion equation (this time for four boundaries instead of two). Since we again have a linear system of equations, we must modify the coefficients and the right-hand side for the equations of the **interior grid points** near the boundaries, depending on the boundary conditions, using the temperatures at the *ghost nodes* as follows (derivation see lecture notes):

**Dirichlet** <br>
*West*
$\begin{equation}
bT_{I^\textrm{S}} - (3a + 2b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - 2aT_{BC}^W
\end{equation}$
*East*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (3a + 2b)T_{I^\textrm{C}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - 2aT_{BC}^E
\end{equation}$
*South*
$\begin{equation}
aT_{I^\textrm{W}} - (2a + 3b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - 2bT_{BC}^S
\end{equation}$
*North*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (2a + 3b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} = -\frac{Q_{I^\textrm{C}}}{k} - 2bT_{BC}^N
\end{equation}$

**Neumann**<br>
*West*
$\begin{equation}
bT_{I^\textrm{S}} - (a + 2b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} + a c^W \Delta{x}
\end{equation}$
*East*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (a + 2b)T_{I^\textrm{C}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} - a c^E \Delta{x}
\end{equation}$
*South*
$\begin{equation}
aT_{I^\textrm{W}} - (2a + b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} + bT_{I^\textrm{N}} = -\frac{Q_{I^\textrm{C}}}{k} + b c^S \Delta{y}
\end{equation}$
*North*
$\begin{equation}
bT_{I^\textrm{S}} + aT_{I^\textrm{W}} - (2a + b)T_{I^\textrm{C}} + aT_{I^\textrm{E}} = -\frac{Q_{I^\textrm{C}}}{k} -b c^N \Delta{y}
\end{equation}$

When initializing the boundary conditions, we make use of the tuple `BC`, which defines the type (`type`) and the value (`val`) of the boundary condition. The temperature at the ghost nodes is then computed later in the script using these values.


In [ ]:
# Boundary Conditions---------------------------------------------------- #
BC      =   (
    type    = (W=:?, E=:?, N=:?, S=:?),
    val     = (W=:?,E=:?,N=:?,S=:?)
)
# ----------------------------------------------------------------------- #

### Solving the Problem

Equation $(5)$ is a linear system of equations of the form

$\begin{equation}
\bm{K} T = T_I,
\end{equation}$

with a coefficient matrix $\bm{K}$ with five non-zero diagonals, the initial temperature condition $T_I$, and the steady-state solution $T$. Therefore, we next define the parameters for our linear system of equations:


In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(?)
K       =   ExtendableSparseMatrix(?)
rhs     =   zeros(?)
# ----------------------------------------------------------------------- #

The assembly of the coefficient matrix, the modification of the right-hand side as described above, and the solution of the linear system are carried out in the function `Poisson!()`. By importing the submodule `GeoModBox.HeatEquation.TwoD`, this function can be called directly (pay attention to the parameters passed to the function!).

In [ ]:
# Solve equation -------------------------------------------------------- #
Poisson2Dc!( ? )
# ----------------------------------------------------------------------- #

Alternatively, the computational steps from the function could be programmed directly here, in which case the submodule would no longer need to be loaded.

## Visualization

Finally, we plot the result:

In [ ]:
# Plot solution --------------------------------------------------------- #
p = heatmap( ? )

contour!( ? )

display(p)

savefig("./Results/04_Steady_State_Solution.png")
# ----------------------------------------------------------------------- #